In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=69ff9ed850e62b2d764c2291b72de4678d3805397a8816668d8386e52de352fe
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/laregion/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

81791


In [4]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    try:
      coverpage= soup.find_all('div',class_='article-header')
      title=coverpage[0].find('h1').get_text()
      # print(title)
    except:
      title=''
    time=''
    try:
      coverpage= soup.find_all('article')
      time=coverpage[0].find('span',class_='content-meta-date').get_text()
      description=coverpage[0].find('div',class_='summary').get_text()
      # print(title)
    except:
      description=''   

    try:
      coverpage = soup.find_all('article')
      article=coverpage[0].find('div',class_='content-body inner-article-data').get_text()
    except:
      article=''
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [5]:
get_articles('https://www.laregion.es/articulo/mundo/futbolista-envia-hermano-boda-acudir-partido/202208081448471151747.html')

['\nUn futbolista envía a su hermano a su boda para acudir a un partido de temporada\n',
 '\n',
 '08/ago./22',
 '\n\nEl delantero de Sierra Leona Mohamed Buya Turay\xa0estaba celebrando su boda cuando\xa0el Malmö anunció su fichaje.\xa0\nEl giro de acontecimientos sucedió cuando el equipo le pidió que acudiese a los entrenamientos, ya que, la temporada en la liga sueca ya había dado comienzo y el conjunto\xa0disputa la previa de competición europea.\nMohamed no canceló su enlace, si no que mandó a su hermano en su lugar, algo tradicional en su país. Sin embargo, el jugador afirmó que en el momento en el que la carga de la temporada no sea tan alta, celebrará nuevamente su boda.\n\n']

In [6]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='La Región'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/laregion/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


url error
error
article error
https://www.laregion.es/video/espanha/blanco-manifestacion-domingo-demasiado-facha-junto-tan-poca-cosa/20190211174617853901.html
article error
https://www.laregion.es/album/mundo/imagenes-atentado-tunez/20150626231916552213.html
article error
https://www.laregion.es/video/mundo/queima-do-gato-vila-flor-vamos-acabar-com-isto/20150626223448552196.html
article error
https://www.laregion.es/video/mundo/declaraciones-rodrigo-rosenberg-parte-i/20090514145455429116.html
article error
https://www.laregion.es/video/mundo/menos-espanoles-mueren-accidente-trafico-peru/20100707111133429980.html
article error
https://www.laregion.es/video/espanha/montero-califica-insulto-mujeres-2019-cuestione-derecho-aborto/20190208191939853337.html
article error
https://www.laregion.es/video/espanha/hallan-cuerpo-descuartizado-mujer-desaparecida-2017/20190208183155853328.html
article error
https://www.laregion.es/video/mundo/espectacular-despegue-boing-787/20150616182513550107.html
a